In [18]:
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model

load_dotenv()

llm = init_chat_model('google_genai:gemini-2.0-flash')

In [19]:
from typing import Annotated 
from langchain_tavily import TavilySearch 
from langchain_core.messages import BaseMessage 
from typing_extensions import TypedDict 

from langgraph.checkpoint.memory import MemorySaver 
from langgraph.graph import StateGraph, START, END 
from langgraph.graph.message import add_messages 
from langgraph.prebuilt import ToolNode, tools_condition 

class State(TypedDict):
    messages : Annotated[list, add_messages] 

graph_builder = StateGraph(State)

tool = TavilySearch(max_results = 2)
tools = [tool] 
llm_with_tools = llm.bind_tools(tool) 

def chatbot(state : State):
    return {'messages':[llm_with_tools.invoke(state['messages'])]}

graph_builder.add_node('chatbot' , chatbot)

tool_node = ToolNode(tools = [tool])
graph_builder.add_node('tools', tool_node) 

graph_builder.add_conditional_edges(
    'chatbot',
    tools_condition
)

graph_builder.add_edge('tools', 'chatbot')
graph_builder.add_edge(START, 'chatbot')

memory = MemorySaver() 
graph = graph_builder.compile(checkpointer=memory)

convert_to_genai_function_declarations expects a Sequence and not a single tool.


In [20]:
config = {'configurable' : {'thread_id' : '1'}}
events = graph.stream(
    {
        "messages" : [
            {
                "role" : "user",
                "content" : (
                    "I'm learning LangGraph."
                    "Could you do some research on it for me?"
                )
            }
        ]
    },
    config,
    stream_mode = "values"
)

for event in events:
    if "messages" in event:
        event['messages'][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph.Could you do some research on it for me?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (7e59211e-2e9a-43cf-b6ae-88da5a4b2fcb)
 Call ID: 7e59211e-2e9a-43cf-b6ae-88da5a4b2fcb
  Args:
    query: LangGraph
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "What is LangGraph? - IBM", "url": "https://www.ibm.com/think/topics/langgraph", "content": "LangGraph, created by LangChain, is an open source AI agent framework designed to build, deploy and manage complex generative AI agent workflows. At its core, LangGraph uses the power of graph-based architectures to model and manage the intricate relationships between various components of an AI agent workflow.

In [21]:
events = graph.stream(
    {
        "messages": [
            {
                "role": "user",
                "content": (
                    "Ya that's helpful. Maybe I'll "
                    "build an autonomous agent with it!"
                ),
            },
        ],
    },
    config,
    stream_mode="values",
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Ya that's helpful. Maybe I'll build an autonomous agent with it!
================================== Ai Message ==================================

That sounds like a great project! LangGraph is designed to help with that. Is there anything else I can help you with regarding LangGraph or autonomous agents?


In [28]:
to_replay = None 
for state in graph.get_state_history(config):
    print('Num Messages: ', len(state.values['messages']), "Next: ", state.next)
    print('-' * 80)
    if len(state.values['messages']) == 3:
        to_replay = state

Num Messages:  6 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  5 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ()
--------------------------------------------------------------------------------
Num Messages:  3 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('tools',)
--------------------------------------------------------------------------------
Num Messages:  1 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  0 Next:  ('__start__',)
--------------------------------------------------------------------------------


In [29]:
print(to_replay.next)
print(to_replay.config)

('chatbot',)
{'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f051809-cc05-6710-8002-5da5222672a4'}}


In [30]:
for event in graph.stream(
    None, 
    to_replay.config, 
    stream_mode = 'values'
):
    if "messages" in event:
        event['messages'][-1].pretty_print()

================================= Tool Message =================================
Name: tavily_search

{"query": "LangGraph", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "What is LangGraph? - IBM", "url": "https://www.ibm.com/think/topics/langgraph", "content": "LangGraph, created by LangChain, is an open source AI agent framework designed to build, deploy and manage complex generative AI agent workflows. At its core, LangGraph uses the power of graph-based architectures to model and manage the intricate relationships between various components of an AI agent workflow. LangGraph illuminates the processes within an AI workflow, allowing full transparency of the agent’s state. By combining these technologies with a set of APIs and tools, LangGraph provides users with a versatile platform for developing AI solutions and workflows including chatbots, state graphs and other agent-based systems. Nodes: In LangGraph, nodes represent individual components or